In [ ]:
import nuclio
import os
from mlrun import get_or_create_ctx

In [7]:
context=get_or_create_ctx('deployf')

[mlrun] 2020-03-09 19:37:12,295 logging run results to: http://10.233.60.111:8080


# NetApp volume mounts

In [ ]:
NETAPP_MOUNT_PATH = context.get_param('NETAPP_MOUNT_PATH')
NETAPP_PVC_CLAIM = context.get_param('NETAPP_PVC_CLAIM')
IGZ_CONTAINER_PATH = context.get_param('IGZ_CONTAINER_PATH')
IGZ_MOUNT_PATH = context.get_param('IGZ_MOUNT_PATH')
spec = nuclio.ConfigSpec(config={"spec.triggers.inference.kind":"cron",
                                "spec.triggers.inference.attributes.interval" :"5m",
                                "spec.readinessTimeoutSeconds" : 360 },
                         env={'FROM_TSDB': 0,
                             'FEATURES_TABLE': context.get_param('FEATURES_TABLE'),
                             'metrics_table': context.get_param('metrics_table'),
                             'PREDICTIONS_TABLE': context.get_param('PREDICTIONS_TABLE'),
                             'MODEL_FILENAME' : context.get_param('MODEL_FILENAME'),
                             'APP_DIR' : context.get_param('APP_DIR'),
                             'DATA_DIR' : str(context.get_input('DATA_DIR')),
                             'SAVE_TO' : context.get_param('SAVE_TO'),
                              'NETAPP_MOUNT_PATH' : NETAPP_MOUNT_PATH
                             },
                         cmd=['pip install pyarrow pandas scikit-learn==0.20.1 dask[dataframe] dask-kubernetes dask-ml["complete"]==1.0.0',
                              'pip install v3io_frames --upgrade',
                             'pip install mlrun'],
                         
                        ).with_v3io().add_volume(IGZ_MOUNT_PATH,IGZ_CONTAINER_PATH,kind='v3io',name='v3io').add_volume(NETAPP_MOUNT_PATH,NETAPP_PVC_CLAIM,kind='pvc',name='netapp')

#addr = nuclio.deploy_file(name='nlp',project='ai',verbose=True, spec=spec, tag='v1.1')

In [ ]:
def handler(context):
    addr = nuclio.deploy_file(source=os.path.join(context.get_param('APP_DIR'),'data-prep.ipynb'),
                                                  name='netops-features',project=context.get_param('PROJECT_NAME'),verbose=True, spec=spec, tag='v1.1')